### HELICITIES

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import mdtraj as md
import seaborn as sns
#from scipy.stats import gaussian_kde
import pandas as pd
from Bio.SeqUtils import IUPACData
from __future__ import print_function
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import itertools
import mdtraj.testing

In [2]:
def calc_helicity(dirname, temp=1,equil=0):
    """
    Calculate the helicity from a trajectory
    """
    topname = os.path.join(dirname, 'prot.pdb')
    trajname = os.path.join(dirname, f'prod{temp}_prot.xtc')
    t = md.load(trajname, top=topname, stride=1)
    t = t[equil:]
    t.remove_solvent(inplace=True)
    #print (t)
    ss = md.compute_dssp(t)
    return ss =='H', ss =='C'

def get_residues(dirname):
    """
    Return the residue list from the topology
    """
    topname = os.path.join(dirname, 'prot.pdb')
    t = md.load(topname, stride=1)
    return list(t.top.residues)

def calc_H_bonds(dirname, temp=1,equil=0):
    """
    Calculate the H_bonds between Q1 and L1.
    In fact this just returns the distance not the H-bond.
    """
    from itertools import product
    topname = os.path.join(dirname, 'prot.pdb')
    trajname = os.path.join(dirname, f'prod{temp}_prot.xtc')
    t = md.load(trajname, top=topname, stride=1)
    t = t[equil:]
    t.remove_solvent(inplace=True)
    print (t)
    #These are the potential donor atom names.
    #Selecting the hydrogens of Q1
    donors = t.topology.select('residue 11 and name HE1 HE2 HE21 HE22')
    if donors.size==0: return np.array([9999.])
    #Selecting the oxigen of L1
    acceptor = t.topology.select('residue 7 and name O')
    pairs = list(product(donors, acceptor))
    d = md.compute_distances(t, pairs)
    #print (d)
    #We take the shortest distance
    d = d.min(1)
    #print (d)
    d *= 10 #to angstroms
    return d

def calc_chis(dirname, temp=1,equil=5000):
    """
    Calculate the Chi1, chi2, chi3 of residue Q1 or equivalent.
    """
    topname = os.path.join(dirname, 'prot.pdb')
    trajname = os.path.join(dirname, f'prod{temp}_prot.xtc')
    t = md.load(trajname, top=topname, stride=1)
    t = t[equil:]
    t.remove_solvent(inplace=True)
    r11 = t.topology.select('residue 11')
    t = t.atom_slice(r11)
    chi1 = md.compute_chi1(t)[1]
    chi2 = md.compute_chi2(t)[1]
    chi3 = md.compute_chi3(t)[1]
    return np.hstack([chi1,chi2,chi3])

def chis11_21(dirname, temp=1,equil=5000):
    """
    Calculate the Chi1, chi2, chi3 of residue Q11 vs Q20 
    """
    topname = os.path.join(dirname, 'prot.pdb')
    trajname = os.path.join(dirname, f'prod{temp}_prot.xtc')
    t = md.load(trajname, top=topname, stride=1)
    t = t[equil:]
    t.remove_solvent(inplace=True)
    r11 = t.topology.select('residue 11')
    tt = t.atom_slice(r11)
    chi1 = md.compute_chi1(tt)[1]
    chi2 = md.compute_chi2(tt)[1]
    chi3 = md.compute_chi3(tt)[1]
    r21 = t.topology.select('residue 21')
    # Esto no seria 'residue 20?'
    tt = t.atom_slice(r21)
    chi11 = md.compute_chi1(tt)[1]
    chi22 = md.compute_chi2(tt)[1]
    chi33 = md.compute_chi3(tt)[1]
    data = np.hstack([chi1,chi2,chi3, chi11, chi22, chi33])
    data *= 180/np.pi
    return pd.DataFrame(data, columns=['chi1_11','chi2_11','chi3_11','chi1_20','chi2_20','chi3_20'])

def plot_helicity(helix, residues, blocks=5, equil=0, **kwargs):
    """
    Plot the helicity from an array of (N,M) of calculated helicities 
    """
    #Remove equilibration
    helix = helix[equil:]
    n_frames = helix.shape[0]
    step = n_frames//blocks
    values = [helix[i*step:(i+1)*step].mean(0) for i in range(blocks)]
    #df=pd.DataFrame(values, columns=[s+str(i) for i,s in enumerate('KKPGASLLLL'+12*'Q'+'KK')]).T
    #df=pd.DataFrame(values, columns=[s for s in 'KKPGASLLLL'+12*'Q'+'KK']).T
    df=pd.DataFrame(values).T
    df =df.unstack().reset_index()
    df.columns=['repetion', 'residue', 'helicity']
    ax=sns.lineplot(x='residue', y='helicity', data=df, sort=False, **kwargs)
    #plt.xticks(np.arange(helix[name].shape[1]), 'KKPGASLLLL'+12*'Q'+'KK')
    #ax.set_xticks(np.arange(helix.shape[1]) )
    #ax.set_xticklabels('KKPGASLLLL'+12*'Q'+'KK')
    ax.grid(True)
    plt.legend()
    plt.ylim(None, 1.)
    ticks=''.join([IUPACData.protein_letters_3to1[r.name.title()] for r in residues])
    plt.xticks(np.arange(helix.shape[1]), ticks)
    
def Q1_helicity(helix, residues, blocks=5, equil=0):
    helix = helix[equil:]
    n_frames = helix.shape[0]
    step = n_frames//blocks
    values = [helix[i*step:(i+1)*step].mean(0) for i in range(blocks)]
    #df=pd.DataFrame(values, columns=[s+str(i) for i,s in enumerate('KKPGASLLLL'+12*'Q'+'KK')]).T
    #df=pd.DataFrame(values, columns=[s for s in 'KKPGASLLLL'+12*'Q'+'KK']).T
    df=pd.DataFrame(values).T
    import statistics
    helicity_Q1 = statistics.mean(df.iloc[10,0:5])
    return helicity_Q1


In [3]:
helix = {}
coil = {}
residues = {},
system_list = ('L4_Q12_a99SB-disp','A4_Q12_a99SB-disp','L4_Q12_DES-amber','A4_Q12_DES-amber','L4_Q12_C36m','A4_Q12_C36m', 'L4_Q12_RSFF', 'A4_Q12_RSFF','L4_Q12_A03ws','A4_Q12_A03ws','L4_Q12_ff15ipq','A4_Q12_ff15ipq','L4_Q12_aFB15','A4_Q12_aFB15')
for system in system_list: 
    #path = f'/home/lourdes/MODULE6/data/{system}'
    path = f'/home/lourdes/TFM-LOURDES/data/ALL/{system}'
    calc_H_bonds(path,equil=0)
    #equil is the equilibration time, which is discarded
    #tupple formed by H (0) and C (1) 
    helix[system], coil[system] = calc_helicity(path, equil=0)
    #Print the length of the trajectory
    print('{:60s} -- {:5d}'.format(path, helix[system].shape[0]))
    residues[system] = get_residues(path)

<mdtraj.Trajectory with 24283 frames, 413 atoms, 24 residues, and unitcells>
/home/lourdes/TFM-LOURDES/data/ALL/L4_Q12_a99SB-disp         -- 24283


TypeError: 'tuple' object does not support item assignment